<a href="https://colab.research.google.com/github/Naomie25/Hackaton-Fashion-Description-Generator/blob/Finale-Version/Fashion_Description_Generator_Hackathon_Last_Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Define the Task & Pipeline Overview

Input (keyword or image) → Generation Model → Quality-Check Module → (Optional) Image Generator → Ethical Filter → Final Output

In [ ]:
!pip install transformers torch sentencepiece
!pip install schedule
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 22.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# ============================
# Installation des bibliothèques (à exécuter une seule fois si besoin)
# ============================
!pip install transformers torch sentencepiece
!pip install schedule

# ============================
# Imports
# ============================
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, BartForConditionalGeneration, BartTokenizer
from transformers import pipeline, set_seed
import difflib
import re
import random

In [ ]:


# ============================
# 1. Configuration générale
# ============================
device = torch.device("cpu")
print("Device set to use", device)

# Générateur texte
generator = pipeline('text-generation', model='distilgpt2', device=-1)
set_seed(42)

# Modèle de résumé (qualité)
bart_model_name = "facebook/bart-base"
bart_tokenizer = BartTokenizer.from_pretrained(bart_model_name)
bart_model = BartForConditionalGeneration.from_pretrained(bart_model_name).to(device)

# Mots clés mode pour le scoring
fashion_keywords = [
    "elegant", "stylish", "refined", "modern", "vintage", "casual",
    "minimalist", "chic", "versatile", "comfort", "premium", "crafted",
    "tailored", "cut", "fit", "fabric", "soft", "bold", "timeless"
]

# ============================
# Génération description produit
# ============================
def generate_descriptions(keyword, num_variants=5):
    prompt = f""" *Item:* {keyword}
 *Description:*
 """

    outputs = generator(prompt, max_new_tokens=120, num_return_sequences=num_variants, temperature=0.75, top_p=0.9,no_repeat_ngram_size=2,
        early_stopping=True,max_length=len(prompt.split()) + 80,)
    print(f'=========\n {outputs}')
    results = []
    for output in outputs:
        gen_text = output["generated_text"]
        # Extract only the generated description part
        description_start = gen_text.find("*Description:*") + len("*Description:*")
        description_text = gen_text[description_start:].strip()
        score = score_description(description_text, prompt)
        results.append((description_text, score))
    results = clean_descriptions(results)
    return results

# ============================
# Résumé qualité (via BART)
# ============================
def summarize_text(text):
    inputs = bart_tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    summary_ids = bart_model.generate(inputs["input_ids"], num_beams=4, max_length=30, early_stopping=True)
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# ============================
# Filtrage éthique
# ============================
def ethical_filter(text):
    blacklist = ["hate", "violence", "racism", "sexism", "terrorism"]
    text_lower = text.lower()
    return not any(bad_word in text_lower for bad_word in blacklist)

# ============================
# Utilitaires : filtrage & score
# ============================
def has_repetitions(text, max_repeat=3):
    pattern = r'\b(\w+)( \1){' + str(max_repeat) + ',}\b'
    return re.search(pattern, text.lower()) is not None

def clean_descriptions(descriptions):
    filtered = []
    for desc, score in descriptions:
        if len(desc.split()) < 8:
            continue
        if has_repetitions(desc):
            continue
        filtered.append((desc, score))
    return filtered

def score_description(desc, prompt):
    words = desc.lower().split()
    keyword_bonus = sum(word in words for word in fashion_keywords)
    length_score = min(len(words), 50) / 50
    similarity = difflib.SequenceMatcher(None, desc.lower(), prompt.lower()).ratio()
    penalty = max(0, 1 - similarity)
    return length_score + 0.5 * keyword_bonus + penalty

# ============================
# Pipeline principal
# ============================
def run_pipeline(keyword, num_variants=5):
    print(f"\n--- Génération pour: {keyword} ---")
    descriptions = generate_descriptions(keyword, num_variants)

    final_results = []
    for desc, score in descriptions:
        summary = summarize_text(desc)
        if not ethical_filter(desc):
            print("❌ Rejeté (filtre éthique):", desc)
            continue
        final_results.append((desc, summary, score))

    for i, (desc, summary, score) in enumerate(final_results, 1):
        print(f"\n✅ Description {i} [Score: {score:.2f}]:\n{desc}")
        print(f"📝 Résumé qualité:\n{summary}")

    generate_image_placeholder()
    return final_results

# ============================
# Placeholder image
# ============================
def generate_image_placeholder():
    print("🖼️ Étape génération image (placeholder)")

# ============================
# Documentation
# ============================
def document_pipeline():
    print("""
Résumé pipeline IA mode (CPU-friendly):
- Génération : distilgpt2
- Résumé qualité : BART-base
- Scoring : mots-clés + longueur + originalité
- Filtre éthique simple
- Image : placeholder
- Utilisation : run_pipeline("mot-clé mode")
""")

# ============================
# Exemple d’utilisation
# ============================
if __name__ == "__main__":
    keyword = "denim jacket"
    run_pipeline(keyword, num_variants=5)
    document_pipeline()

Device set to use cpu


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=120) and `max_length`(=84) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



--- Génération pour: denim jacket ---
 [{'generated_text': ' *Item:* denim jacket\n *Description:*\n ******************************\n\n"We\'re going to be in a very good shape in the next couple weeks. We\'ll be working on an awesome jacket, but it won\'t be enough for us to get to the point where we\'re not going anywhere.\n* * *\nThe jacket has been in use for a long time now, so we are excited about it. It\'s our second color, and we will be able to wear it for the rest of the year. The jacket was last used for some time in 2011, when it was the only color that was available. In 2011 we'}, {'generated_text': ' *Item:* denim jacket\n *Description:*\n \xa0\n* The denim is a denim band jacket with a pair of jeans with the sleeves of the denim and a jacket. This jacket has been worn with both the jacket and the leather jacket, but it has only been fitted with one belt.\nThe denim has a two-piece waistband and is the only one to be worn. The jeans are white, a black and black, and have 